In [4]:
import pandas as pd
import numpy as np

import json
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 100000000)
import warnings



In [3]:
df = pd.read_csv('input_etl_data.csv', sep = ';')

In [5]:
# 1. Extract three new columns from column via 

# Change to string and the to json
# Preprocess double quotes for json valid format

df['via'] = df['via'].apply(lambda row: json.loads(row.replace('“', '"').replace('”', '"')))

# Extract desired columns

df['channel'] = df['via'].apply(lambda row: row['channel'] if 'channel' in row else '')
df['address_client'] = df['via'].apply(lambda row: row['source']['from']['address'] if 'address' in row['source']['from'] else '')
df['name_client'] = df['via'].apply(lambda row: row['source']['from']['name'] if 'name' in row['source']['from'] else '')

# Drop via column
df.drop(['via'], axis='columns', inplace=True)

In [8]:
from src.preprocessing import feature_engineering
fe = feature_engineering.Preprocess()

In [9]:
# Day of the week and week of the year column
df = fe.datetime_transform(df, 'created_at')

In [11]:
# Create group_des column 
df.group_id.dtype
# Verify data type
df['group_id'] = df.group_id.astype('object')

In [12]:
# Set up conditions
conditions = [
    (df['group_id'] == 20938601),
    (df['group_id'] == 21052033),
    (df['group_id'] == 21054107),
    (df['group_id'] == 360002394054),
    (df['group_id'] == 28479838),
    (df['group_id'] == 360002386813),
    (df['group_id'] == 21406268)

]
# set up decode words to match
values = ['Soporte', 'Francia', 'Italia', 'EN', 'Portugual', 'DE', 'Postventa']

df['group_des'] = np.select(conditions, values)

In [21]:
df.to_csv('output_etl_data.csv')